In [ ]:
import pandas as pd
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait

driver = webdriver.Chrome("")
url = "https://www.jobkorea.co.kr/recruit/joblist?menucode=duty"

driver.get(url)
# 잡코리아 홈페이지 접속
headers = {'User-Agent': 'Mozilla/5.0'}
res = requests.get(driver.current_url, headers = headers)
# 차단되었을때 우회하는 방법

driver.find_element(By.XPATH, value = '//*[@id="devSearchForm"]/div[2]/div/div[1]/dl[1]/dd[2]/div[2]/dl[1]/dd/div[1]/ul/li[6]').click()
# 개발,데이터 분야 클릭

driver.find_element(By.XPATH, value = '//*[@id="duty_step2_10031_ly"]/li[1]').click()
driver.find_element(By.XPATH, value = '//*[@id="duty_step2_10031_ly"]/li[2]').click()
driver.find_element(By.XPATH, value = '//*[@id="duty_step2_10031_ly"]/li[4]').click()
driver.find_element(By.XPATH, value = '//*[@id="duty_step2_10031_ly"]/li[5]').click()
driver.find_element(By.XPATH, value = '//*[@id="duty_step2_10031_ly"]/li[8]').click()
driver.find_element(By.XPATH, value = '//*[@id="duty_step2_10031_ly"]/li[9]').click()
# 원하는 직무 선택

driver.find_element(By.XPATH, value = '//*[@id="devCndtDispArea"]/div/dl[1]/dd[2]').click()

time.sleep(3)
# Wait = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "dev-gi-list")))
# 선택된 데이터로 직업 검색 클릭

company_name_list = []
Career_list = []
academic_list = []
Location_list = []
Type_list = []
Salary_list = []
Title_list = []
company_url_list = []

def list_select():
    body = driver.find_element(By.ID, value = "dev-gi-list")
## 원하는 body태그
    tr = body.find_elements(By.CLASS_NAME, value = 'devloopArea')
## body안의 tr태그
    for tr_ in tr:
        name = tr_.find_elements(By.CSS_SELECTOR, value = 'a')
        company_name_list.append(name[0].text)
## 기업 이름 리스트 뽑기

        title = tr_.find_elements(By.CSS_SELECTOR, value = 'strong')
        Title_list.append(title[0].text)
## 공고문 제목 리스트 뽑기

        herf = tr_.find_element(By.TAG_NAME, value = 'a')
        company_url_list.append(herf.get_attribute('href'))
## 해당 공고문에 대한 url 링크 리스트 뽑기

    # Wait = WebDriverWait(driver, 10).until(EC.presence_of_elements_located((By.CLASS_NAME,'cell')))
        info = tr_.find_elements(By.CLASS_NAME, value = 'cell')
        for i in range(len(info)):
            if i == 0:
                Career_list.append(info[i].text)
            elif i == 1:
                academic_list.append(info[i].text)
            elif i == 2:
                Location_list.append(info[i].text)
            elif i == 3:
                Type_list.append(info[i].text)
            # elif len(info) != 5:
            #     Salary_list.append('')
            # elif i == 4:
            #     Salary_list.append(info[i].text)

                    # data가 없는 부분이 있어 잠시 생략

# table에서 class가 cell에 해당하는 정보 뽑기
# 경력, 학력, 위치 등 정보 리스트 뽑기

def data_setup():
    data = {'Title' : Title_list,
        'Company' : company_name_list,
        'Career' : Career_list,
        'academic' : academic_list,
        'Location' : Location_list,
        'Type' : Type_list,
        # 'Salary' : Salary_list,
        'URL' : company_url_list
       }
    job_df = pd.DataFrame(data)

    return job_df

    #데이터프레임으로 만들기


def page_Move(page):
    page_div = driver.find_element(By.ID, value = "dvGIPaging")
    page_li = page_div.find_elements(By.TAG_NAME, value = "li")
    page_p = page_div.find_elements(By.TAG_NAME, value = "p")

    if page != 0 and ((page+1)%10) == 0:
                for p in page_p:
                    a_tag = p.find_elements(By.TAG_NAME, value = "a")
                    for a in a_tag:
                        if "다음" in a.text:
                            a.click()
    elif page_li[page] is None:
        condition = False
        return condition
    
    elif page_li[page].text == str(page+1):
        page_li[page+1].click()
        # print(page_li[page].text)

        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "tbody")))
        # 새로운 페이지 클릭하고 새로운 요소가 생성될때까지 대기(최대 10초)
# 페이지 이동 함수

page = 0 
# 페이지 리스트 초기값
condition = True

while condition:
    list_select()
    page_Move(page)
    page += 1

#     if page == 10:
#         condition = False

#페이지 이동을 위해 while문 반복문 사용

data_setup().to_excel('job_list.xlsx',index=False)